In [355]:
import pickle
import warnings
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime
import json
import time
from abc import ABC, abstractmethod
from dataclasses import dataclass
from collections import namedtuple, defaultdict
from typing import Optional, Any, Union, List
from copy import deepcopy

#### Задача данного иследования - собрать инфу о дополнительныйх параметрах, разобраться с сущностями, доступных каждым эндпоинтом и написать базовые функции проверки параметров (валидации). По результату накатаем часть функционала нашей либы.

#### Подтянем результаты и классы из ноута с изучением api

In [2]:
root_url = "http://iss.moex.com"
docs_url = "/iss/reference"
save_path = 'moex_api_response'

In [9]:
@dataclass
class MoexApi:
    url_faq: str
    url_template: str
    description: str = ''

In [269]:
period = ['yearly', 'monthly', 'daily']

In [12]:
class Globals:
    __instance = None
    globals_url = "https://iss.moex.com/iss/index.json"
    url_index_id = "https://iss.moex.com/iss/statistics/engines/stock/markets/index/analytics.json"
    
    base_attr = namedtuple("base_attr", "main description")
    
    main_attribute = {
        "engines": base_attr("name", "title"),
        "markets": base_attr("market_name", "market_title"),
        "boards": base_attr("boardid", "board_title"),
        "boardgroups": base_attr("name", "title"),
        "durations": base_attr("interval", "title"),
        "securitytypes": base_attr("security_type_name", "security_type_title"),
        "securitygroups": base_attr("name", "title"),
        "securitycollections": base_attr("name", "title"),
    }
    datatypes = ["securities", "trades"]
    
    report_names = pd.DataFrame(
        data=[
            [
                "numtrades", 
                "Информация о количестве договоров по инструментам, "
                "являющимся производными финансовыми инструментами (по валютным парам)",
            ],
            [
                "participants",
                "Информация о количестве лиц, имеющих открытые позиции по инструментам, "
                "являющимся производными финансовыми инструментами (по валютным парам)",
            ],
            [
                "openpositions",
                "Информация об открытых позициях по инструментам, являющимся производными "
                "финансовыми инструментами (по валютным парам)",
            ],
            [
                "expirationparticipants",
                "Информация о количестве лиц, имеющих открытые позиции по договорам, "
                "являющимся производными финансовыми инструментами (по срокам экспирации)",
            ],
            [
                "expirationopenpositions",
                "Информация об объеме открытых позиций по договорам, являющимся производными "
                "финансовыми инструментами (по срокам экспирации)",
            ],
        ],
        columns=['report_name', 'decription']
    )
    report_names.set_index('report_name', inplace=True)
    
    sessions = pd.DataFrame(
        data=[
            [1, "Основная сессия"], [2, "Вечерняя сессия"], [3, "Итого (все сессии)"], [0, "Утренняя сессия"]
        ], 
        columns=['sessions', 'decription']
    )
    sessions.set_index('sessions', inplace=True)
    
    
    def __new__(cls, *args, **kwargs):
        if not cls.__instance:                
            cls.__instance = super().__new__(cls, *args, **kwargs)
        return cls.__instance
    
    def __init__(self):
        moex_dict = requests.get(self.globals_url).json()
        for entity, columns in self.main_attribute.items():
            setattr(self, entity, pd.DataFrame(
                data=moex_dict[entity]['data'],
                columns=moex_dict[entity]['columns'],
                ).set_index(columns.main)
            )
        self.indexids = self.get_index_id()
    
    def get_index_id(self):
        index_ids = requests.get(self.url_index_id).json()
        df_index_ids = pd.DataFrame(
            data=index_ids['indices']['data'], columns=index_ids['indices']['columns']
        )
        df_index_ids.set_index('indexid', inplace=True)
        return df_index_ids
    
    
    def description(self, entity: str, name):
        entities = entity + "s"
        use_entity = getattr(self, entities, None)
        if use_entity is None:
            raise NameError(f"Entity '{entity}' is not found!")
        if not isinstance(use_entity, pd.DataFrame):
            raise ValueError(f"Entity '{entity}' hasn't description")
        return getattr(self, entity + "s").loc[name, (self.main_attribute[entities].description,)]
            

In [13]:
with open('api.pickle', 'rb') as file:
    data = pickle.load(file)
    
OK_API = data['use_api']  # API, доступные без подписки (FYI)
GLOBAL_GUID = data['globals']  # Глобальный справочник сущностей (FYI)

In [37]:
display(GLOBAL_GUID.boards.head())
print(list(OK_API.items())[:5])

,id,board_group_id,engine_id,market_id,board_title,is_traded,has_candles,is_primary
boardid,,,,,,,,
TQIF,177,57,1,1,Т+: Паи - безадрес.,1,1,1
TQTF,178,57,1,1,Т+: ETF - безадрес.,1,1,1
TQBR,129,57,1,1,Т+: Акции и ДР - безадрес.,1,1,1
TQBS,130,57,1,1,Т+: А2-Акции и паи - безадрес.,0,1,1
TQNL,131,57,1,1,Т+: Б-Акции и паи - безадрес.,0,1,1


[(5, MoexApi(url_faq='http://iss.moex.com/iss/reference/5', url_template='http://iss.moex.com/iss/securities', description='Список бумаг торгуемых на московской бирже.')), (24, MoexApi(url_faq='http://iss.moex.com/iss/reference/24', url_template='http://iss.moex.com/iss/turnovers', description='Получить сводные обороты по рынкам.Например: https://iss.moex.com/iss/turnovers.xml')), (100, MoexApi(url_faq='http://iss.moex.com/iss/reference/100', url_template='http://iss.moex.com/iss/turnovers/columns', description='Получить описание полей для запросов оборотов по рынку/торговой системе.\rНапример: https://iss.moex.com/iss/engines/stock/turnovers/columns.xml')), (28, MoexApi(url_faq='http://iss.moex.com/iss/reference/28', url_template='http://iss.moex.com/iss/index', description='Получить глобальные справочники ISS. \rНапример: https://iss.moex.com/iss/index.xml')), (40, MoexApi(url_faq='http://iss.moex.com/iss/reference/40', url_template='http://iss.moex.com/iss/engines', description='Пол

In [28]:
@dataclass
class ApiParams:
    key: str
    description: Optional[str] = None
    default: Optional[str] = None
    type_: Optional[str] = None

## Соберем все параметры запроса к апи:

In [244]:
API_PARAMS = {}
param_key = None
for api_id, api in OK_API.items():
    API_PARAMS[api_id] = {}
    req = requests.get(api.url_faq)
    soup = bs(req.content)
    
    for data in soup.body.dl:
        if data.name is None:
            continue

        if data.name == 'dt':
            data_entity = data.get_text()
            API_PARAMS[api_id][data_entity] = {}
            continue
        
        if data.dl is None:
            continue
        
        API_PARAMS[api_id][data_entity]['description'] = data.find('pre').get_text()

        for params in data.dl:
            if params.name == 'dt':
                if param_key is None:
                    param_key = params.get_text()
                    continue
                else:
                    raise ValueError("BAD ERROR")
                    
            if params.name == 'dd':
                description = params.find('pre').get_text()
                default, type_ = params.find_all('strong')

                API_PARAMS[api_id][data_entity].setdefault('params', []).append(
                    ApiParams(
                        key = param_key,
                        description = description,
                        default = default.next.next.text,
                        type_ = type_.next.next.text,
                    )
                )
                param_key = None

    time.sleep(.2)

In [245]:
print(len(API_PARAMS) == len(OK_API))
list(API_PARAMS.items())[:3]

True


[(5,
  {'securities (GET)': {'description': 'Список бумаг торгуемых на московской бирже.',
    'params': [ApiParams(key='q', description='Поиск инструмента по части Кода, Названию, ISIN, Идентификатору Эмитента, Номеру гос.регистрации.\nНапример: https://iss.moex.com/iss/securities.xml?q=MOEX\nСлова длиной менее трёх букв игнорируются. Если параметром передано два слова через пробел. То каждое должно быть длиной не менее трёх букв.', default='', type_='var'),
     ApiParams(key='lang', description='Язык результата: ru или en', default='ru', type_='var'),
     ApiParams(key='engine', description='', default='', type_='var'),
     ApiParams(key='is_trading', description='', default='', type_='var'),
     ApiParams(key='market', description='', default='', type_='var'),
     ApiParams(key='group_by', description='Группировать выводимый результат по полю. Доступны значения group и type.', default='', type_='var'),
     ApiParams(key='limit', description='Количество выводимых инструментов (

#### Убедимся что все ок, почистим данные, отберем уникальные параметры

1. Удалим все (GET) из ключа возвращаемой сущности
2. Проверим глазками (выборочно) поинты не принимающие уточняющие параметры
3. Отберем уникальные ключи параметров запроса

In [251]:
# 1. rename data
for key, data in API_PARAMS.items():
    for data_type in list(data.keys()):
        if data_type.find(' (GET)') == -1:
            print(key, data_type)
        else:
            data[data_type.replace(" (GET)", "")] = data.pop(data_type)

5 securities


In [256]:
# 2. find all data_type without params
for key, data in API_PARAMS.items():
    for data_type, value in data.items():
        if value.get('params') is None:
            print(OK_API[key].url_faq)
            print(data_type)
            break

http://iss.moex.com/iss/reference/843
analytics.columns
http://iss.moex.com/iss/reference/123
changeover
http://iss.moex.com/iss/reference/172
coefficients.dates
http://iss.moex.com/iss/reference/168
cbrf
http://iss.moex.com/iss/reference/758
splits
http://iss.moex.com/iss/reference/165
data.dates
http://iss.moex.com/iss/reference/166
data.dates
http://iss.moex.com/iss/reference/169
dates
http://iss.moex.com/iss/reference/134
securities.dates
http://iss.moex.com/iss/reference/171
quotedsecurities.dates
http://iss.moex.com/iss/reference/933
monthend_accints.index
http://iss.moex.com/iss/reference/195
aggregates.dates
http://iss.moex.com/iss/reference/159
issuecapitalization
http://iss.moex.com/iss/reference/161
boards
http://iss.moex.com/iss/reference/711
securities.dates
http://iss.moex.com/iss/reference/716
history.dates
http://iss.moex.com/iss/reference/214
agregates.dates
http://iss.moex.com/iss/reference/634
params.dates
http://iss.moex.com/iss/reference/41
timetable
http://iss.moe

### Проверил, все ок!

In [260]:
UNIQUE_KEY = {}
types_var = set()
for data in API_PARAMS.values():
    for data_type in data.values():
        for param in data_type.get('params', []):
            name = param.key
            types_var.add(param.type_)
            UNIQUE_KEY.setdefault(name, {"description": set(), "default": set()})
            UNIQUE_KEY[name]["description"].add(param.description)
            UNIQUE_KEY[name]["default"].add(param.default)
print(types_var)

{'number', 'var', 'string'}


In [265]:
UNIQUE_KEY

{'q': {'description': {'Поиск инструмента по части Кода, Названию, ISIN, Идентификатору Эмитента, Номеру гос.регистрации.\nНапример: https://iss.moex.com/iss/securities.xml?q=MOEX\nСлова длиной менее трёх букв игнорируются. Если параметром передано два слова через пробел. То каждое должно быть длиной не менее трёх букв.'},
  'default': {''}},
 'lang': {'description': {'', 'Язык результата: ru или en'},
  'default': {'ru'}},
 'engine': {'description': {'',
   'Показывать типы инструментов для торговой системы.'},
  'default': {''}},
 'is_trading': {'description': {''}, 'default': {''}},
 'market': {'description': {'',
   'Рынок\nEQ - индекс акций\nFI - индекс облигаций\nMX - составные индексы\n'},
  'default': {''}},
 'group_by': {'description': {'Группировать выводимый результат по полю. Доступны значения group и type.'},
  'default': {''}},
 'limit': {'description': {'',
   'Количество выводимых бумаг (20,100)',
   'Количество выводимых бумаг доступны значения (1, 5, 10, 20, 50, 100)'

Сейчас перейдем к части подготовки чек-функций для доступных параметров

In [299]:
# 1. Выпилим параметры, которые считаем безсполезными
UNUSE_PARAMS = {
    "lang", "is_tonight_session", "sort_order_desc", "table_type", "show", "security_type_id", 
    "seqnum", "format", "group_by_filter",
}

In [275]:
# Этот поинт генерит ссылки на данные доступные только подписчикам
API_PARAMS.pop(116, None)
OK_API.pop(116, None)

In [300]:
for param in UNUSE_PARAMS:
    UNIQUE_KEY.pop(param, None)

In [301]:
for point in API_PARAMS.values():
    for entity in point.values():
        params = entity.get("params")
        if params is None:
            continue
        for idx in range(len(params)-1, -1, -1):
            if params[idx].key in UNUSE_PARAMS:
                params.pop(idx)

In [389]:
UNIQUE_KEY['tickers']

{'description': {''}, 'default': {''}}

In [391]:
# Напишем миксин проверки 
# Для поиска значения в множестве можно создать метод - генерирующий функцию проверки (аля DRY), но для удобства 
# сопровождения будем писать отдельный метод под каждое множество.

ENDPOINT_DEFAULTS = {
    873: {"limit": 500},
    791: {"numtrades": 1},
    147: {"limit": 100},
}

ALLOWED_MANY = {"securities", "boardid", "assets", "sectypes"}


CHECK_METHODS = {
    'q': "check_instrument_find",
    'engine': "check_engine",
    'is_trading': "check_bool",
    'market': "check_market",
    'group_by': "check_group_by",
    'limit': "check_limit",
    'start': "check_start",
    'date': "check_date",
    'is_traded': "check_bool",
    'hide_inactive': "check_bool",
    'securitygroups': "check_security_group",
    'trade_engine': "check_engine",
    'time': "check_time",
    'asset_type': "check_asset_type",
    'sort_order': "check_sort_order",
    'tradingsession': "check_session",
    'security_collection': "check_security_collection",
    'type': "check_type",
    'latest': "check_bool",
    'only_actual': "check_bool",
    'securities': "check_securities",
    'boardid': "check_boards",
    'from': "check_date",
    'till': "check_date",
    'status': "check_status",
    'numtrades': "check_int",
    'interim': "check_bool",
    'assetcode': "check_instrument_find",
    'sort_column': "check_sort_columns",
    'primary_board': "check_bool",
    'assets': "check_assets",
    'index': "check_bool",
    'previous_session': "check_bool",
    'first': "check_int",
    'leaders': "check_bool",
    'nearest': "check_bool",
    'sectypes': "check_sectype",
    'tradeno': "check_int",
    'reversed': "check_bool",
    'recno': "check_bool",
    'next_trade': "check_bool",
    'yielddatetype': "check_yielddatetype",
    'interval': "check_interval",
    'iss.reverse': "check_bool_like_bool",
    'year': "check_int",
    'month': "check_int",
    'expiration_date': "check_date",
    'option_type': "check_option_type",
    'series_type': "check_option_series_type",
    'tickers': "check_instrument_find",
}

class MoexParamCheckerMixin:
    SECTYPE = {
        '1': 'Акция обыкновенная',
        '2': 'Акция привилегированная',
        '3': 'Государственные облигации',
        '4': 'Региональные облигации',
        '5': 'Облигации центральных банков',
        '6': 'Корпоративные облигации',
        '7': 'Облигации МФО',
        '8': 'Биржевые облигации',
        '9': 'Паи открытых ПИФов',
        'A': 'Паи интервальных ПИФов',
        'B': 'Паи закрытых ПИФов',
        'C': 'Муниципальные облигации',
        'D': 'Депозитарные расписки',
        'E': 'Бумаги иностранных инвестиционных фондов (ETF)',
        'F': 'Ипотечный сертификат',
        'G': 'Корзина бумаг',
        'H': 'Доп. идентификатор списка',
        'I': 'ETC (товарные инструменты)',
        'J': 'Пай биржевого ПИФа (Exchange Investment Unit share)'
    }
    OPTION_SERIES_TYPE = {
        "D": "дневной",
        "W": "недельный",
        "M": "месячный", 
        "Q": "квартальный",
    }
    
    current_endpoint_id: int = -1
    
    # Добавить в основной класс
    set_engine: set = set()
    set_security_group: set = set()
    set_session: set[int] = set()
    set_security_collection: set = set()
    set_board: set = set()
    set_security_type: set = set()
    set_duration: set[int] = set()
        
    set_market: set = {"EQ", "FI", "MX"}
    set_group_by: set = {"group", "type"}
    set_asset_type: set = {"S", "F"}
    set_sort_order: set = {"asc", "desc"}
    set_type: set = {"daily", "monthly"}
    set_status = {"traded", "nottraded", "all"}
    set_yielddatetype = {"MBS", "MATDATE", "OFFERDATE"}
    set_option_type = {"C", "P"}

    
    @abstractmethod
    def get_endpoint_columns(self) -> set: 
        """
        У многих эндпоинтов есть сортировка по столбцу. Столбцы мы уже получили в json формате ранее.
        Подготовим их чуть позже и сохраним в отдельном модуле.
        """

    @staticmethod
    def check_instrument_find(q: Any):
        if not isinstance(q, str):
            q = str(q)
        for word in q.split():
            if len(word) < 3:
                raise ValueError("Запрос инструментов длиной менее трёх букв игнорируются.")
        return q
                
    def check_engine(self, engine: str):
        if engine not in self.set_engine:
            raise ValueError(f"Engine: '{engine}' не найден")
        return engine
    
    @staticmethod
    def check_bool(val: Any):
        if val:
            return 1
        return 0
    
    def check_market(self, market: str):
        upper_market = market.upper()
        if upper_market not in self.set_market:
            raise ValueError("Доступны: EQ - индекс акций, FI - индекс облигаций, MX - составные индексы")
        return upper_market
    
    def check_group_by(self, group_by: str):
        lower_group_by = group_by.lower()
        if lower_group_by not in self.set_group_by:
            raise ValueError("Доступны значения group и type")
        return lower_group_by
    
    @staticmethod
    def check_limit(self, limit: int):
        if not isinstance(limit, int):
            raise ValueError("Лимит должен быть целым числом")
        return limit
    
    @staticmethod
    def check_start(self, start: int):
        if not isinstance(limit, int):
            raise ValueError("Курсор должен быть целым числом")
        return start
    
    @staticmethod
    def check_date(value_date: Union[str, datetime.date, datetime.datetime]):
        if isinstance(value_date, (datetime.date, datetime.datetime)):
            return value_date.strftime(value_date, '%Y-%m-%d')
        try:
            datetime.datetime.strptime(value_date, '%Y-%m-%d')
            return value_date
        except:
            raise ValueError(f"Значение даты {value_date} не является датой или не удовлетворяет формату ГГГГ-ММ-ДД")
            
    def check_security_group(self, security_group: str):
        if security_group not in self.set_security_group:
            raise ValueError(f"Группа {security_group} не найдена.")
        return security_group
    
    @staticmethod
    def check_time(value_time: Union[str, datetime.time, datetime.datetime]):
        if isinstance(value_time, (datetime.time, datetime.datetime)):
            return value_time.strftime(value_time, '%H:%M:%S')
        try:
            datetime.datetime.strptime(value_time, '%H:%M:%S')
            return value_time
        except:
            raise ValueError(f"Значение времени {value_time} не удовлетворяет формату ЧЧ:ММ:CC")
    
    def check_asset_type(self, asset_type: str):
        upper_asset_type = asset_type.upper()
        if upper_asset_type not in self.set_asset_type:
            raise ValueError("Доступны фильтры по типу базового актива. (S - Опционы на акцию, F - Опционы на фьючерс)")
        return upper_asset_type
    
    
    def check_sort_order(self, sort_order: str):
        lower_sort_order = sort_order.lower()
        if lower_sort_order not in self.set_sort_order:
            raise ValueError('Направление сортировки. "asc" - По возрастанию значения, "desc" - По убыванию!')
        return lower_sort_order
    
    def check_session(self, session: Union[str, int]):
        if isinstance(session, str):
            if session.isdigit():
                session = int(session)
        if session in self.set_session:
            return session
        raise ValueError('Укажите корректную сессию: 0 - Утренняя;  1 - Основная;  2 - Вечерняя;  3 - Итого')
        
    def check_security_collection(self, security_collection: str):
        if security_collection not in self.set_security_collection:
            raise ValueError(f"Группа ФИ '{security_collection}' не найдена.")
        return security_collection
    
    def check_type(self, value_type: str):
        lower_value_type = value_type.lower()
        if lower_value_type not in self.set_type:
            raise ValueError("Не верный тип капитализации. Доступные значения: daily, monthly")
        return lower_value_type
    
    @staticmethod
    def check_securities(securities: Union[str, List[str]], max_security=10):
        if isinstance(securities, list) and len(securities) > max_security:
            raise ValueError(f"Запросить можно не более {max_security} фин. инструментов")
        return securities
    
    def check_boards(self, boards: Union[str, List[str]]):
        if isinstance(boards, str):
            boards = [boards]
        for board in boards:
            if board not in self.set_board:
                raise ValueError(f"Площадка '{board}' не найдена.")
        return boards
    
    def check_status(self, status: str):
        lower_status = status.lower()
        if lower_status not in self.set_status:
            raise ValueError(f"Ошибка cтатуса. Укажите фильтр торгуемости инструментов: traded, nottraded или all")
        return lower_status
    
    @staticmethod
    def check_int(value: Union[str, int]):
        if isinstance(value, int) or (isinstance(value, str) and value.isdigit()):
            return value
        raise ValueError(f"Передан признак [{value}], который должен быть целым числом!!! ")
        
    def check_sort_columns(self, column):
        available_columns: set = self.get_endpoint_columns()
        if column not in available_columns:
            raise ValueError(f"Не найден атрибут сортировки {column} в доступных {sorted(available_columns)}")
        return column
    
    def check_assets(self, securities: Union[str, List[str]]):
        return self.check_securities(securities, 5)
    
    def check_sectype(self, values: Union[str, List[str]]):
        warnings.warn(
            """
            Поле 'sectypes' не соответствует значениям из глобального справочника. Для ПФИ указывается краткий код БА,
            например, si, ri, mx и т.д. Для спота обратитесть к справочнику 'SECTYPE' объекта.
            """
        )
        if isinstance(values, list) and len(values) > 5:
            raise ValueError(f"Запросить можно не более 5 типов фин. инструментов")
        elif isinstance(values, str):
            values = [values]
        for value in values:
            if len(value) > 1:
                continue
            if value not in self.SECTYPE:
                raise ValueError(f"Код {value} не найден в справочнике типов фин. инструментов")
        return values
        
    def check_yielddatetype(self, yielddatetype: str):
        upper_yielddatetype = yielddatetype.upper()
        if upper_yielddatetype not in self.set_yielddatetype:
            raise ValueError(f"Фильтр доступен по типам доходности: MBS, MATDATE, OFFERDATE")
        return upper_yielddatetype
    
    def check_interval(self, interval: int):
        if interval not in self.set_duration:
            raise ValueError("Интервал должен соответствовать доступным значениям. Смотрите справочник 'durations'.")
        return interval
    
    def check_bool_like_bool(self, value: Any):
        if self.check_bool(value):
            return "true"
        return "false"
    
    def check_option_type(self, option_type: str):
        upper_option_type = option_type.upper()
        if upper_option_type not in self.set_option_type:
            raise ValueError("Не верный тип опциона. C - CALL, P - PUT")
        return upper_option_type
    
    def check_option_series_type(self, series_type: str):
        upper_series_type = series_type.upper()
        if upper_series_type not in self.OPTION_SERIES_TYPE:
            raise ValueError("Не найдена серия опциона. Обратитесь к справочнику OPTION_SERIES_TYPE")
        return upper_series_type
    

#### Миксин готов. Соберем поля, возвращаемые каждым поинтом:

In [396]:
all_columns = {}
path = "moex_api_response"

def join_columns(api_id):
    all_columns[api_id] = d = {}
    with open(f"{path}/{api_id}.json", "r") as file:
        data = json.load(file)
    for key, value in data.items():
        d[key] = value["columns"]

        
for key in OK_API.keys():
    join_columns(key)

In [398]:
# В указанных поинтах косяки, не определны колонки. Остальное ок.
all_columns[769]['netflow2'] = []
all_columns[809]['futoi'] = []

Часть эндпоинтов возвращает курсор, а чать - нет, хотя и в них присутствует пагинация. Для нашей либы нужно знать, что это за эндпроинты.
Дополнительно проверим атрибутный состав курсоров.

In [402]:
api_cursor = {}
cursor_columns = set()
for api_id, api_resp in all_columns.items():
    for entity in api_resp:
        if entity.find('cursor') != -1:
            api_cursor[api_id] = entity
            cursor_columns.add(tuple(api_resp[entity]))
            break
    if api_id in api_cursor:
        continue
        
    entites_params = API_PARAMS[api_id]
    for values_params in entites_params.values():
        params = values_params.get('params', [])
        for param in params:
            if param.key in {'limit', 'start'}:
                api_cursor[api_id] = None
                break
        if api_id in api_cursor:
            break

In [403]:
api_cursor

{5: None,
 873: None,
 172: 'coefficients.cursor',
 165: 'data.cursor',
 134: 'securities.cursor',
 649: None,
 191: 'sitenews.cursor',
 193: 'events.cursor',
 162: 'securities.cursor',
 13: None,
 89: None,
 118: None,
 119: None,
 813: 'history.cursor',
 817: 'history.cursor',
 821: 'history.cursor',
 815: None,
 55: None,
 35: None,
 56: None,
 155: None,
 157: None,
 46: None,
 34: None,
 62: 'history.cursor',
 791: None,
 63: 'history.cursor',
 793: None,
 64: 'history.cursor',
 795: None,
 65: None,
 797: None,
 131: 'securities.cursor',
 147: 'analytics.cursor',
 715: 'history.cursor',
 712: 'securities.cursor'}

In [404]:
cursor_columns

{('INDEX', 'TOTAL', 'PAGESIZE'),
 ('INDEX', 'TOTAL', 'PAGESIZE', 'PREV_DATE', 'NEXT_DATE')}

Мы собрали список ключей курсоров (для контролируемых запросов). И поинты по которым курсор недоступен.

In [418]:
for key, val in OK_API.items():
    if not val.description:
        print(key, val)

In [417]:
OK_API[205].description = 'Описание колонок бумаг входящих в индекс'
OK_API[165].description = 'РЕПО ГЦБ объемы (УСТАРЕЛО)'
OK_API[166].description = 'РЕПО ГЦБ детали (УСТАРЕЛО)'
OK_API[196].description = 'Описание колонок по агрегированным показетелям рынка бондов'
OK_API[178].description = 'Ставки РЕПО аукционов (ГЦБ)'
OK_API[179].description = 'Описание полей для запроса ставок РЕПО аукционов (ГЦБ)'
OK_API[767].description = 'Сводная статистика по клиентам с самым высоким нетто-потоком. Детали https://fs.moex.com/f/10374/netflow2-demo-en.html'
OK_API[807].description = 'Сводные остатки по фьючерсам на физ. и юр. лица.'
OK_API[634].description = 'Кривые безкупонной доходности'
OK_API[156].description = 'Интервалы свечей, доступных к запросу по ФИ'
OK_API[769].description = (
    'Сводная статистика по клиентам с самым высоким нетто-потоком. Запрос возможен с лагом -15 дней от текущего для 1 эшелона.'
    'Детали https://fs.moex.com/f/10374/netflow2-demo-en.html'
)
OK_API[809].description = 'Сводные остатки по фьючерсам на физ. и юр. лица. Запрос возможен с лагом -15 дней, по коду фьючерса'

В данном разделе осталось последнее - создать общий справочник для нашей либы. Предлагаю сделать в универсальном формате - JSON. Он должен содержать всю инфу, требуемую для предоставления пользователю описаний api и метаинформации для формирования запросов.
Что он должен содержать:
1. id API
2. Описание эндпоинта
3. url FAQ
4. Шаблон API
5. Глобальные ключи сущностей
6. Список возвращаемых API данных и их описание
7. Колонки возвращаемых данных
8. Ключи параметров запроса, их описание и дефолтное значение
9. Наличие курсоров

**Пример структуры:**
```
$id: {
    description: str 
    faq_url: str
    endpoint: str
    global_entities: list[$entity_name] | None
    has_cursor: bool | None
    cursor_name: str | None
    return_data: {
        $data_name: {
            description: str
            columns: list[str]
        }
    }
    params: {
        $key_name: {
            description: str
            default: str | None
        }
    }
}
```

In [432]:
FINAL_JSON = {}
for key, value in OK_API.items():
    FINAL_JSON[key] = d = {}
    d['description'] = value.description
    d['faq_url'] = value.url_faq
    d['endpoint'] = value.url_template
    global_entities = [entity[1:-1] for entity in re.findall(r'{\w*}', value.url_template)]
    if global_entities:
        d['global_entities'] = global_entities
    if key in api_cursor:
        d['has_cursor'] = True
        d['cursor_name'] = api_cursor[key]
    api_params = API_PARAMS[key]
    params = dict()
    return_data = dict()
    for data_entity, data_value in api_params.items():
        return_data[data_entity] = {
            "description": data_value.get('description', ''),
            "columns": all_columns[key].get(data_entity, []),
        }
        for param_key in data_value.get('params', []):
            if param_key.key not in params:
                params[param_key.key] = {
                    "description": param_key.description,
                    "default": param_key.default,
                }
                
    d['return_data'] = return_data
    d['params'] = params
        
    

In [438]:
with open("MOEX_API_DICT.json", "w", encoding='utf-8') as file:
    json.dump(FINAL_JSON, file, ensure_ascii=False)

#### На этом сбор данных об апи закончен. Мы собрали нужные данные для разработки библиотеки быстрой работы с API MOEX. Идеи разработки ниже. В следующей части мы протестим уже ее работу.



1. Заведение общих справочников
2. Заведение глобальных сущностей
3. Подготовка методов поиска ФИ и торгуемых площадок
4. Инструмент для работы с сущностями
5. Возрват доступных АПИ и их описание
6. Запросы по апи с параметрами
7. Проверки параметров при запросе
8. Запросы с курсорами
9. Запросы с лимитами без курсоров
10. Обновление справочника

Писать буду в IDE (Pycharm)